# **When and Why to Use setUpClass and tearDownClass**

In [1]:
# Import necessary libraries
import unittest

# Step 1: Define the Book class
class Book:
    def __init__(self, title, price, discount):
        self.title = title
        self.price = price
        self.discount = discount

    def apply_discount(self):
        if not 0 <= self.discount <= 1:
            raise ValueError("Discount must be between 0 and 1")
        return self.price * (1 - self.discount)

# Step 2: Create Test Class
class TestBook(unittest.TestCase):
    @classmethod
    def setUpClass(cls):
        print("Setting up class resources...")
        # This book instance will be used across multiple tests
        cls.book = Book("Python Testing", 50.0, 0.2)

    @classmethod
    def tearDownClass(cls):
        print("Tearing down class resources...")
        # Clean up the book instance
        del cls.book

    def test_apply_discount_valid(self):
        """Test apply_discount with a valid discount"""
        discounted_price = self.book.apply_discount()
        self.assertEqual(discounted_price, 50.0 * 0.8)  # 20% off 50.0 should be 40.0

    def test_apply_discount_zero(self):
        """Test apply_discount with zero discount"""
        self.book.discount = 0
        discounted_price = self.book.apply_discount()
        self.assertEqual(discounted_price, 50.0)

    def test_apply_discount_full(self):
        """Test apply_discount with a full discount (100%)"""
        self.book.discount = 1
        discounted_price = self.book.apply_discount()
        self.assertEqual(discounted_price, 0.0)

    def test_apply_discount_invalid(self):
        """Test apply_discount with an invalid discount"""
        self.book.discount = -0.5  # Invalid discount
        with self.assertRaises(ValueError):
            self.book.apply_discount()

# Step 3: Run the tests in Google Colab
unittest.main(argv=[''], verbosity=2, exit=False)


test_apply_discount_full (__main__.TestBook)
Test apply_discount with a full discount (100%) ... ok
test_apply_discount_invalid (__main__.TestBook)
Test apply_discount with an invalid discount ... ok
test_apply_discount_valid (__main__.TestBook)
Test apply_discount with a valid discount ... ERROR
test_apply_discount_zero (__main__.TestBook)
Test apply_discount with zero discount ... ok

ERROR: test_apply_discount_valid (__main__.TestBook)
Test apply_discount with a valid discount
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-1-de64fef3897c>", line 32, in test_apply_discount_valid
    discounted_price = self.book.apply_discount()
  File "<ipython-input-1-de64fef3897c>", line 13, in apply_discount
    raise ValueError("Discount must be between 0 and 1")
ValueError: Discount must be between 0 and 1

----------------------------------------------------------------------
Ran 4 tests in 0.017s

FAILED (errors=

Setting up class resources...
Tearing down class resources...


Explanation

    Book Class Definition:
        apply_discount calculates a discounted price based on the discount attribute.
        It raises a ValueError if discount is outside the 0-1 range, ensuring robust error handling.

    TestBook Class:
        setUpClass initializes a Book instance once before any tests run.
        tearDownClass deletes the Book instance after all tests are complete, ensuring no lingering resources.
        Test Methods:
            test_apply_discount_valid: Verifies a 20% discount on the initial price.
            test_apply_discount_zero: Checks that a zero discount doesn’t change the price.
            test_apply_discount_full: Ensures that a 100% discount reduces the price to zero.
            test_apply_discount_invalid: Tests for an invalid discount value and expects a ValueError.

    Running the Tests:
        unittest.main(argv=[''], verbosity=2, exit=False) is used to run the tests in Google Colab, with verbosity=2 for detailed output and exit=False to prevent Colab from terminating the cell after tests.

Expected Output
The output should show each test running with setup and teardown logs:

Example Scenario: Mocking an API Call

Imagine a class WeatherService that fetches weather data from an external API. Testing this class directly would depend on the availability and response of the actual API. Instead, we can mock the API request to return a controlled, predictable response.
Step-by-Step Code Example

    Define the WeatherService Class: This class includes a method that makes an HTTP request to a weather API.
    Create a Test Class Using Mocks: We’ll use patch to replace the actual API call with a mock during testing.

# **Define the WeatherService Class**

In [2]:
import requests

class WeatherService:
    def __init__(self, api_key):
        self.api_key = api_key

    def get_weather(self, city):
        # Simulating an HTTP request to a weather API
        response = requests.get(f"https://api.weather.com/v3/weather/{city}?apiKey={self.api_key}")

        if response.status_code == 200:
            return response.json()
        else:
            return None


The get_weather method in WeatherService uses requests.get to fetch weather data for a given city.
If the response status is 200 OK, it returns the JSON data; otherwise, it returns None.

# Step 2: Create a Test Class Using Mocks and Patching

Now, let's write tests for get_weather without actually making HTTP requests. We’ll patch requests.get to mock the API response.

In [3]:
import unittest
from unittest.mock import patch, Mock

class TestWeatherService(unittest.TestCase):
    @patch('requests.get')  # Patching 'requests.get' in our test
    def test_get_weather_success(self, mock_get):
        # Mock response object with a 200 OK status and JSON data
        mock_response = Mock()
        mock_response.status_code = 200
        mock_response.json.return_value = {"temp": 72, "description": "sunny"}

        # Set the mock_get return value to our mock_response
        mock_get.return_value = mock_response

        # Initialize WeatherService and call get_weather
        service = WeatherService(api_key="dummy_api_key")
        result = service.get_weather("New York")

        # Assertions
        self.assertEqual(result, {"temp": 72, "description": "sunny"})
        mock_get.assert_called_once_with("https://api.weather.com/v3/weather/New York?apiKey=dummy_api_key")

    @patch('requests.get')
    def test_get_weather_failure(self, mock_get):
        # Mock response object with a 404 Not Found status
        mock_response = Mock()
        mock_response.status_code = 404

        # Set the mock_get return value to our mock_response
        mock_get.return_value = mock_response

        # Initialize WeatherService and call get_weather
        service = WeatherService(api_key="dummy_api_key")
        result = service.get_weather("New York")

        # Assertions
        self.assertIsNone(result)
        mock_get.assert_called_once_with("https://api.weather.com/v3/weather/New York?apiKey=dummy_api_key")

# Run the tests in Google Colab or Jupyter Notebook
unittest.main(argv=[''], verbosity=2, exit=False)


test_apply_discount_full (__main__.TestBook)
Test apply_discount with a full discount (100%) ... ok
test_apply_discount_invalid (__main__.TestBook)
Test apply_discount with an invalid discount ... ok
test_apply_discount_valid (__main__.TestBook)
Test apply_discount with a valid discount ... ERROR
test_apply_discount_zero (__main__.TestBook)
Test apply_discount with zero discount ... ok
test_get_weather_failure (__main__.TestWeatherService) ... ok
test_get_weather_success (__main__.TestWeatherService) ... ok

ERROR: test_apply_discount_valid (__main__.TestBook)
Test apply_discount with a valid discount
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-1-de64fef3897c>", line 32, in test_apply_discount_valid
    discounted_price = self.book.apply_discount()
  File "<ipython-input-1-de64fef3897c>", line 13, in apply_discount
    raise ValueError("Discount must be between 0 and 1")
ValueError: Discount must be be

Setting up class resources...
Tearing down class resources...


Explanation of Each Part

    Using @patch('requests.get') Decorator:
        The @patch decorator replaces requests.get with a mock object mock_get for the duration of the test method. This prevents any actual HTTP request from being made.

    Mocking Successful Response (test_get_weather_success):
        We create a mock response (mock_response) with:
            status_code set to 200 (indicating a successful HTTP response).
            json.return_value set to a dictionary {"temp": 72, "description": "sunny"}, simulating a JSON response from the API.
        mock_get.return_value = mock_response tells mock_get to return mock_response whenever requests.get is called.
        Assertion:
            self.assertEqual(result, {"temp": 72, "description": "sunny"}): Checks that the method returns the expected mock JSON.
            mock_get.assert_called_once_with(...): Ensures requests.get was called with the correct URL.

    Mocking Failed Response (test_get_weather_failure):
        We create another mock_response, this time with status_code set to 404 (not found).
        The test asserts that get_weather returns None when a non-200 status code is received.
        Assertion:
            self.assertIsNone(result): Checks that the method returns None for an unsuccessful API call.

    Running the Tests:
        unittest.main(argv=[''], verbosity=2, exit=False) runs the tests with more detailed output.